In [1]:
import torch
import torch.nn as nn

In [2]:
device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
device

device(type='cuda')

In [3]:
!wget https://github.com/devic1/RNN-/raw/main/data.zip

--2023-02-11 16:30:17--  https://github.com/devic1/RNN-/raw/main/data.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/devic1/RNN-/main/data.zip [following]
--2023-02-11 16:30:17--  https://raw.githubusercontent.com/devic1/RNN-/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23197 (23K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  22.65K  --.-KB/s    in 0s      

2023-02-11 16:30:18 (58.9 MB/s) - ‘data.zip’ saved [23197/23197]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/boys.txt           
  inflating: data/girls.txt          


In [5]:
import torch 
import torch.nn as nn
import os

In [6]:
with open("data/boys.txt","r") as f:
  data = f.read()
data = data.split("\n")

In [7]:
data[:5]

['அகண்டலன்', 'அகத்தியன்', 'அகரன்', 'அகரமுதல்வன்', 'அகற்கண்ணன்']

In [8]:
lett = set()
for i in data:
  for j in i:
    lett.add(j)
lett.add(".")

In [9]:
ltoi = {i:k for k,i in enumerate(sorted(lett))}
ll_n = len(ltoi)
print(ltoi)
ll_n

{' ': 0, ',': 1, '.': 2, ';': 3, 'ஃ': 4, 'அ': 5, 'ஆ': 6, 'இ': 7, 'உ': 8, 'ஊ': 9, 'எ': 10, 'ஏ': 11, 'ஐ': 12, 'ஒ': 13, 'ஓ': 14, 'க': 15, 'ங': 16, 'ச': 17, 'ஜ': 18, 'ஞ': 19, 'ட': 20, 'ண': 21, 'த': 22, 'ந': 23, 'ன': 24, 'ப': 25, 'ம': 26, 'ய': 27, 'ர': 28, 'ற': 29, 'ல': 30, 'ள': 31, 'ழ': 32, 'வ': 33, 'ஷ': 34, 'ஸ': 35, 'ஹ': 36, 'ா': 37, 'ி': 38, 'ீ': 39, 'ு': 40, 'ூ': 41, 'ெ': 42, 'ே': 43, 'ை': 44, 'ொ': 45, 'ோ': 46, 'ௌ': 47, '்': 48}


49

In [10]:
itol = {l:k for k,l in ltoi.items()}
print(itol)

{0: ' ', 1: ',', 2: '.', 3: ';', 4: 'ஃ', 5: 'அ', 6: 'ஆ', 7: 'இ', 8: 'உ', 9: 'ஊ', 10: 'எ', 11: 'ஏ', 12: 'ஐ', 13: 'ஒ', 14: 'ஓ', 15: 'க', 16: 'ங', 17: 'ச', 18: 'ஜ', 19: 'ஞ', 20: 'ட', 21: 'ண', 22: 'த', 23: 'ந', 24: 'ன', 25: 'ப', 26: 'ம', 27: 'ய', 28: 'ர', 29: 'ற', 30: 'ல', 31: 'ள', 32: 'ழ', 33: 'வ', 34: 'ஷ', 35: 'ஸ', 36: 'ஹ', 37: 'ா', 38: 'ி', 39: 'ீ', 40: 'ு', 41: 'ூ', 42: 'ெ', 43: 'ே', 44: 'ை', 45: 'ொ', 46: 'ோ', 47: 'ௌ', 48: '்'}


In [11]:
class RNN(nn.Module):
  def __init__(self,inp_size,hid_size,out_size):
    super().__init__()
    self.inp_size = inp_size
    self.hid_size = hid_size
    self.out_size = out_size
    self.i2h = nn.Linear(self.inp_size+self.hid_size,self.hid_size)
    self.i2o = nn.Linear(self.inp_size+self.hid_size,self.out_size)
    self.o2o = nn.Linear(self.hid_size+self.out_size,self.out_size)
    self.drop = nn.Dropout(0.1)
    self.softmax = nn.LogSoftmax(dim=1)
  
  def forward(self,inp,hid):
    combined_out = torch.cat((inp,hid),dim=1)
    hid_1 = self.i2h(combined_out)
    out = self.i2o(combined_out)
    combined_sec = torch.cat((hid_1,out),1)
    x = self.o2o(combined_sec)
    x = self.drop(x)
    x = self.softmax(x)
    return x,hid_1

  def temp_hidden(self):
    return torch.zeros(1,self.hid_size)


In [45]:
rnn = RNN(ll_n,128,ll_n).to(device)

In [13]:
def one_enc(lett):
  return torch.nn.functional.one_hot(torch.tensor(ltoi[lett]),num_classes=ll_n).unsqueeze(0)

In [14]:
lente = len(data)
def getrandinp():
  randn = torch.randint(0,lente,(1,))
  inp = torch.tensor([])
  out = torch.tensor([],dtype=torch.long)
  word = data[randn]
  for i,j in zip(word,word[1:]+"."):
    tens = one_enc(i)
    inp = torch.cat((inp,tens),0)
    teno = torch.tensor(ltoi[j]).unsqueeze(0)
    out = torch.cat((out,teno))
  return inp,out.unsqueeze(1),word

In [59]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(),lr=0.005)

In [82]:
epoch = 100000
rnn.train()
for num in range(epoch):
  hid = rnn.temp_hidden()
  inp,tar,word = getrandinp()
  inp,tar,hid = inp.to(device),tar.to(device),hid.to(device)
  loss = 0
  for i in range(len(word)):
    out,hid = rnn(inp[i].unsqueeze(0),hid)
    loss += loss_fn(out,tar[i])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if num%(epoch/10) == 0:
    print(loss/len(word))

tensor(1.8813, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.2253, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.8658, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.5741, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.4960, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.0438, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.7527, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.5321, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3960, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6496, device='cuda:0', grad_fn=<DivBackward0>)


In [91]:
rnn.eval()
with torch.no_grad():
  for i in range(10):
    #t = input("Enter a continuous letter : ")
    t = itol[torch.randint(5,ll_n,(1,)).item()]
    hid = rnn.temp_hidden()
    hid = hid.to(device)
    real_out = ""
    while True:
      n = one_enc(t)
      n = n.to(device)
      out,hid = rnn(n,hid)
      _,result = out.topk(3)
      res = result[0][torch.randint(0,3,(1,))].item()
      real_out += t
      if itol[res] == ".":
        break
      t = itol[res]
    print(real_out)

ஹன் 
ொலைம்
ஹாட்டிய்முபழகுரி
ஐனிரன்பன் அறுமைபஙபிருமி
ஊமுரசு
மாரியாமி
உத்சுநந்குறிவார்பன்பி
த்ராம்
இளஞன்யன்
ஊருஜி


In [85]:
torch.save(rnn.state_dict(), "/content/drive/MyDrive/college/rnn_model.pt")

In [86]:
print("Happy Ending :)")

Happy Ending :)
